In [64]:
#Cameron R. Shabahang
#Risk Management

import pandas as pd
import numpy as np
import scipy.stats as stat
import matplotlib.pyplot as plt
import math
import scipy as sp
from sklearn.decomposition import PCA

#Choose 10 stocks and 1 year historical daily prices
df1 = pd.read_csv('JPM.csv', dtype = object)
df2 = pd.read_csv('GS.csv', dtype = object)
df3 = pd.read_csv('MS.csv', dtype = object)
df4 = pd.read_csv('C.csv', dtype = object)
df5 = pd.read_csv('CS.csv', dtype = object)
df6 = pd.read_csv('UBS.csv', dtype = object)
df7 = pd.read_csv('JEF.csv', dtype = object)
df8 = pd.read_csv('LAZ.csv', dtype = object)
df9 = pd.read_csv('BCS.csv', dtype = object)
df10 = pd.read_csv('EVR.csv', dtype = object)

#Form a portfolio with chosen weights and shares
def filter(df):
    z = df[['Date','Adj Close']]
    close = z['Adj Close'].astype(float)
    dailyreturns = close.pct_change(1)
    z.insert(2,"Daily Return",dailyreturns)
    z['Daily Px Change'] = close - close.shift(1)
    z = z.iloc[1:]
    z['Weighted'] = z['Daily Return'].apply(lambda x: x*0.1)
    return z

df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = [filter(df) for df in (df1, df2, df3, df4, df5, df6, df7, df8, df9, df10)]
portfolio_return = df1['Weighted']+df2['Weighted']+df3['Weighted']+df4['Weighted']+df5['Weighted']+ df6['Weighted']+df7['Weighted']+df8['Weighted']+df9['Weighted']+df10['Weighted']
print(portfolio_return)

#Calculate 99% historical VaR for the portfolio
portfolioVaRobservation = round(0.01 * len(portfolio_return))
portfoliohistoricalVaR = sorted(portfolio_return)[portfolioVaRobservation]*100
print('The 99% historical VaR is:',portfoliohistoricalVaR,'%')



1     -0.008636
2     -0.005588
3     -0.000178
4     -0.003326
5      0.009783
         ...   
248   -0.032210
249   -0.003011
250   -0.047456
251   -0.015656
252    0.045301
Name: Weighted, Length: 252, dtype: float64
The 99% historical VaR is: -9.204044592969312 %


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
#Calculate 99% parametric VaR for the portfolio
def pVar(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10):
    weights = np.repeat(0.10, 252)
    #df_wMu = 0.1*((np.mean(df1['Daily Return'])+np.mean(df2['Daily Return'])+np.mean(df3['Daily Return'])+np.mean(df4['Daily Return'])+ #Needs to take df1-df10
                  #np.mean(df4['Daily Return'])+np.mean(df5['Daily Return'])+np.mean(df6['Daily Return'])+np.mean(df7['Daily Return'])+
                  #np.mean(df8['Daily Return'])+np.mean(df9['Daily Return'])+np.mean(df10['Daily Return']))
    frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
    df_merge = pd.concat(frames, axis=1)
    z=df_merge['Daily Return']
    print(z)
    covar = np.cov(df_merge['Daily Return'])
    pvar = np.dot(weights.T,np.dot(covar,weights)) #Takes covariance of df1-df10
    portfolioParametricVaR = 2.326*np.sqrt(pvar)
    return portfolioParametricVaR
print('The 95% parametric portfolio VaR is:', pVar(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10))



     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  \
1       -0.007349     -0.009572     -0.014712     -0.009666     -0.011765   
2        0.001234     -0.006362     -0.005468     -0.008325     -0.002976   
3        0.001232     -0.002248      0.001269      0.001592     -0.010448   
4       -0.001670     -0.017635      0.000634     -0.005347     -0.004525   
5        0.000528      0.004288     -0.001900     -0.001453      0.008333   
..            ...           ...           ...           ...           ...   
248     -0.044473     -0.027570     -0.036514     -0.015398     -0.024803   
249     -0.027396     -0.005302      0.007074     -0.026992      0.009249   
250     -0.049319      0.001627     -0.036628     -0.056363     -0.073310   
251     -0.038110     -0.008290     -0.001042     -0.054596     -0.032138   
252      0.061834      0.008953      0.000261      0.083045      0.047893   

     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  

In [63]:
#Portfolio 0.95 decay SD for VaR

decay_mean_shift_a = np.zeros(252)
decay_mean = np.zeros(252)
decay_var = np.zeros(252)
for i in range (len(portfolio_return)):
    rollingreturn_shift = portfolio_return[0:i-1]
    rollingreturn = portfolio_return[0:i]
    decay_mean[i] = np.mean(rollingreturn)
    decay_mean_shift_a[i] = np.mean(rollingreturn_shift)
    decay_mean_shift = decay_mean_shift_a[2:]
    decay_var[i] = 0.95*decay_var[i-1]+(1-0.95)*(decay_mean_shift[i] - decay_mean[i])**2
print(decay_mean)
print(decay_mean_shift)
print(decay_var)

IndexError: index 250 is out of bounds for axis 0 with size 250

In [ ]:
Calculate 99% Historical PnL VaR for the portfolio
positionsize = 100000
sharesize_c = positionsize/float(rawdata.iloc[0,5])
sharesize_d = positionsize/float(rawdata2.iloc[0,5])
historicalportfolioPnLVaR = sorted(finaldata['Daily Px Change'])[VaRobservation]*sharesize_d + sorted(finaldata2['Daily Px Change'])[VaRobservation]*sharesize_c
print('The 95% historical portfolio PnL VaR is: $', historicalportfolioPnLVaR)

#Calculate 95% Parametric PnL VaR for the portfolio
#d_Mu = np.mean(finaldata['Daily Px Change'])
#d_Sigma = np.std(finaldata['Daily Px Change'])
#c_Mu = np.mean(finaldata2['Daily Px Change'])
#c_Sigma = np.std(finaldata2['Daily Px Change'])
#p_Mu = 0.5 * d_Mu + 0.5 * c_Mu
#covar = np.cov(finaldata['Daily Px Change'],finaldata2['Daily Px Change'])[0,1]
#p_Sigma = np.sqrt(((0.5*0.5)*(c_Sigma*c_Sigma))+2*(0.5*0.5)*covar*(c_Sigma*d_Sigma)+(0.5*0.5)*(d_Sigma*d_Sigma))
# = (p_Mu - 1.645 * p_Sigma) * (sharesize_c + sharesize_d)
#print('The 95% parametric portfolio PnL VaR is: $', parametricportfolioPnLVaR)

In [ ]:
#Implement Factor Model VaR
#Run principle component analysis and retrieve the first two factors
#95% of the eigenvalues
#Locate the 1% position 

pca = PCA(n_components=10)
principalComponents = pca.fit_transform(x)
principalDf = pd.z(data = principalComponents)
principalDf



In [116]:
#HW2 EVT, EWMA(GARCH), and MC for Parametric Portfolio VaR
#EVT in VaR
#Find values for u and nu
port_SD = pVar(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10)/2.326
u = 1.645*port_SD
order = portfolio_return.sort_values(ascending=False)
order = order.reset_index(drop=True)
print(order)   
i=0
while(order[i]>u):
    i=i+1
evt = order[0:i]
print('EVT list is:\n', evt)
n = len(portfolio_return)
a = 0.05
print('The value of u is: ', u)
print('The value of nu is: ', i)



     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  \
1       -0.007349     -0.009572     -0.014712     -0.009666     -0.011765   
2        0.001234     -0.006362     -0.005468     -0.008325     -0.002976   
3        0.001232     -0.002248      0.001269      0.001592     -0.010448   
4       -0.001670     -0.017635      0.000634     -0.005347     -0.004525   
5        0.000528      0.004288     -0.001900     -0.001453      0.008333   
..            ...           ...           ...           ...           ...   
248     -0.044473     -0.027570     -0.036514     -0.015398     -0.024803   
249     -0.027396     -0.005302      0.007074     -0.026992      0.009249   
250     -0.049319      0.001627     -0.036628     -0.056363     -0.073310   
251     -0.038110     -0.008290     -0.001042     -0.054596     -0.032138   
252      0.061834      0.008953      0.000261      0.083045      0.047893   

     Daily Return  Daily Return  Daily Return  Daily Return  Daily Return  

In [ ]:
#Estimate xi and beta
def est(xi, beta):
  y=(1+(xi*(evt-u)/beta))^(-1/(xi-1))
  return(-(1/beta)*sum(log(y)))
estimate=mle(est,start=list(xi = 1.5, beta = 1.5))
xi=as.double(estimate@coef[1]) 
beta=as.double(estimate@coef[2]) 
print(xi)
print(beta)

#Solve for u*
ustar<-u+(beta/xi)*(((n/nu)*(a))^(-xi)-1)
print(ustar)

#Simulate 1 day distribution
pca=princomp(returns)
eigvals<-pca$sdev^2
explained<-rep(0, length(eigvals))
for(i in 1:length(eigvals))
  explained[i]=eigvals[i]/sum(eigvals)

#95% test
nfacs95=0
esum95=0
while(esum95<0.95):
  esum95=esum95+explained[nfacs95+1]
  nfacs95=nfacs95+1

#6 factors required for 95%
varcov<-cov(pca$loadings)
facmean<-mean(pca$loadings)

#2 Days of Prices with EWMA
#window = 60 or 3 months
lambda=0.97
n=2
set.seed(-1000)
errors=rnorm(1000, 0, 1)
rets=rep(0, n+1)
prices=rep(0, n+1)
sigsq=rep(0, n+1)
eps=rep(0, n+1)
rets[1]=mean(portRet) #set to simulated mean
prices[1]=1000000
sigsq[1]=sd(portRet)^2 #set to simulated vol
eps[1]=sqrt(sigsq)*errors[1]
for(i in range 2:(n+1)):
  sigsq[i]=lambda*sigsq[i-1]^2+(1-lambda)*rets[i-1]^2
  eps[i]=np.sqrt(sigsq[i])*errors[i]
  rets[i]=eps[i]+lambda*rets[i-1]+(1-lambda)*eps[i-1]
  prices[i]=(1+rets[i])*prices[i-1])